In [1]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

2025-12-24 17:51:34.491471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766598694.676550      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766598694.732709      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766598695.186794      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766598695.186831      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766598695.186834      55 computation_placer.cc:177] computation placer alr

In [2]:
df = pd.read_csv('/kaggle/input/02-brad-features/02_brad_features.csv')

In [3]:
# Load the emotion detection model with GPU support
emotion_classifier = pipeline("text-classification", model="hatemnoaman/bert-base-arabic-finetuned-emotion", device=0)

# Initialize the emotion columns
emotion_labels = ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']
for label in emotion_labels:
    df[label] = 0.0

# Function to get emotion scores for each review
def get_emotion_scores(text):
    # Truncate the text to 512 tokens
    truncated_text = text[:512]
    results = emotion_classifier(truncated_text)
    scores = {result['label']: result['score'] for result in results}
    return [scores.get(label, 0.0) for label in emotion_labels]

# Apply the function to your dataset with progress bar
emotion_scores = df['review_clean'].apply(get_emotion_scores)

# Use tqdm to show progress
for i, label in enumerate(emotion_labels):
    print(f"Processing {label}")
    df[label] = [scores[i] for scores in tqdm(emotion_scores, desc=f"Processing {label}")]


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

Processing love: 100%|██████████| 154007/154007 [00:00<00:00, 3158566.79it/s]


Processing sadness


Processing sadness: 100%|██████████| 154007/154007 [00:00<00:00, 3159880.13it/s]


Processing surprise


Processing surprise: 100%|██████████| 154007/154007 [00:00<00:00, 3159339.21it/s]


In [4]:
df.columns

Index(['Unnamed: 0', 'rating', 'review_id', 'book_id', 'user_id', 'review',
       'review_clean', 'camel_sentiment', 'camel_score', 'gt_sentiment',
       'rating_normalized', 'sentiment_score', 'final_score', 'len_chars',
       'len_words', 'num_exclam', 'num_qmark', 'len_chars_norm',
       'len_words_norm', 'num_exclam_norm', 'num_qmark_norm', 'anger', 'fear',
       'joy', 'love', 'sadness', 'surprise'],
      dtype='object')

In [5]:
# Load the aspect-based sentiment model
aspect_classifier = pipeline("text-classification", model="aubmindlab/bert-base-arabertv02", device=0)

# Function to get aspect sentiment scores for each review
def get_aspect_sentiment_scores(text):
    # Truncate the text to 512 tokens
    truncated_text = text[:512]
    results = aspect_classifier(truncated_text)
    scores = {result['label']: result['score'] for result in results}
    return scores

# Apply the function to your dataset
aspect_sentiment_scores = df['review_clean'].apply(get_aspect_sentiment_scores)

# Add aspect sentiment scores to your dataset
for label in aspect_sentiment_scores[0].keys():
    df[label] = 0.0

for i, scores in enumerate(aspect_sentiment_scores):
    for label in scores.keys():
        df.loc[i, label] = scores[label]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
df = df.drop(columns=['LABEL_0', 'LABEL_1'])
df.columns

Index(['Unnamed: 0', 'rating', 'review_id', 'book_id', 'user_id', 'review',
       'review_clean', 'camel_sentiment', 'camel_score', 'gt_sentiment',
       'rating_normalized', 'sentiment_score', 'final_score', 'len_chars',
       'len_words', 'num_exclam', 'num_qmark', 'len_chars_norm',
       'len_words_norm', 'num_exclam_norm', 'num_qmark_norm', 'anger', 'fear',
       'joy', 'love', 'sadness', 'surprise'],
      dtype='object')

In [9]:
df.to_csv('02_brad_features_increasing.csv', index=False)